In [44]:
# create model class
import torch
import torch.nn as nn

In [ ]:
class Model(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features,3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid(),
    )


  def forward(self, features):
    out = self.network(features)

    return out

  # def loss(self,y)

In [ ]:
# create features
features = torch.rand(10,5)

# create model
model = Model(num_features=features.shape[1])

#call model for forward pass
#model.forward(features)
model(features)


tensor([[0.5400],
        [0.5407],
        [0.5487],
        [0.5365],
        [0.5310],
        [0.5428],
        [0.5402],
        [0.5328],
        [0.5489],
        [0.5431]], grad_fn=<SigmoidBackward0>)

In [ ]:
model.network[0].weight

Parameter containing:
tensor([[-0.0188, -0.3322,  0.1676, -0.1719, -0.0571],
        [-0.4267, -0.0590, -0.2268, -0.2949,  0.4037],
        [ 0.3659, -0.4320,  0.0225, -0.0970,  0.0345]], requires_grad=True)

In [ ]:
model.network[0].bias

Parameter containing:
tensor([ 0.3595, -0.4377,  0.3360], requires_grad=True)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model,input_size=(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [43]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [45]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")

df.drop(columns=["id","Unnamed: 32"], inplace=True)

In [46]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [47]:
x_train,x_test,y_train,y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

In [48]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [49]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [50]:
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [113]:
class MySimpleNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()

    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()

    # self.linear = nn.Sequential(
    #     nn.Linear(num_features,1),
    #     # nn.ReLU(),
    #     # nn.Linear(3,1),
    #     nn.Sigmoid()
    # )

  def forward(self, features):
      out = self.linear(features)
      out = self.sigmoid(out)

      return out




In [118]:
learning_rate = 0.1
epochs = 50

In [115]:
loss_function = nn.BCELoss()

In [119]:
model = MySimpleNN(x_train_tensor.shape[1])

#Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):
  # forward
  y_pred = model(x_train_tensor.float())

  # loss
  loss = loss_function(y_pred, y_train_tensor.float().view(-1,1))
  # print(f"Epoch: {epoch+1}, loss: {loss.item()}")

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update with no gradient
  optimizer.step()


  print(f"Epoch: {epoch+1}, loss: {loss.item()}")

Epoch: 1, loss: 0.5813307762145996
Epoch: 2, loss: 0.46151813864707947
Epoch: 3, loss: 0.39515361189842224
Epoch: 4, loss: 0.3529715836048126
Epoch: 5, loss: 0.3233853280544281
Epoch: 6, loss: 0.30119770765304565
Epoch: 7, loss: 0.28375130891799927
Epoch: 8, loss: 0.2695457637310028
Epoch: 9, loss: 0.2576678395271301
Epoch: 10, loss: 0.2475278377532959
Epoch: 11, loss: 0.2387268841266632
Epoch: 12, loss: 0.23098449409008026
Epoch: 13, loss: 0.2240971475839615
Epoch: 14, loss: 0.21791306138038635
Epoch: 15, loss: 0.21231640875339508
Epoch: 16, loss: 0.2072170376777649
Epoch: 17, loss: 0.20254355669021606
Epoch: 18, loss: 0.19823846220970154
Epoch: 19, loss: 0.19425496459007263
Epoch: 20, loss: 0.19055432081222534
Epoch: 21, loss: 0.18710429966449738
Epoch: 22, loss: 0.18387764692306519
Epoch: 23, loss: 0.18085119128227234
Epoch: 24, loss: 0.1780051440000534
Epoch: 25, loss: 0.17532232403755188
Epoch: 26, loss: 0.1727878302335739
Epoch: 27, loss: 0.17038863897323608
Epoch: 28, loss: 0.16

In [120]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(x_test_tensor.float())
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.5036934614181519
